In [10]:
import pandas as pd
import sys
sys.path.insert(0, "..")
from random_utility import BradleyTerry
from task import ChoiceTask
from sklearn.model_selection import train_test_split
import pickle
from collections import Counter

In [2]:
with open('../examples/data/product_choices.pickle', 'rb') as handle:
    choice_data = pickle.load(handle)
    
with open('../examples/data/product_info.pickle', 'rb') as handle:
    product_data = pickle.load(handle)

In [3]:
choice_data.head()

,STORE_ID,BASKET_ID,choice,alternatives
0,286,29483925362,[6396581],"[5582789, 5584007, 951703, 6396581]"
1,286,29773175255,[5584007],"[5582789, 5584007, 951703, 6396581]"
2,286,30673440294,[5584007],"[5582789, 5584007, 951703, 6396581]"
3,286,32269485930,[951703],"[5582789, 5584007, 951703, 6396581]"
4,286,40876776136,[5582789],"[5582789, 5584007, 951703, 6396581]"


In [4]:
product_data.head()

,PRODUCT_ID,prod_size,price,price_per_size
0,849098,7.0,1.021950,0.145993
1,951703,4.0,0.568998,0.142249
2,5578643,6.0,0.393425,0.065571
3,5582789,6.0,0.392923,0.065487
4,5584007,6.0,0.393564,0.065594


In [5]:
train, test = train_test_split(choice_data, random_state=42, test_size=0.01)

In [6]:
products_bought_train = ChoiceTask(choice_data, 'alternatives', 'choice', features_to_use=['price_per_size', 'prod_size'], secondary_table=product_data,
                             secondary_to_primary_link={"PRODUCT_ID": ['alternatives', 'choice']})
# products_bought_test = ChoiceTask(test, 'alternatives', 'choice', features_to_use=['price_per_size', 'prod_size'], secondary_table=product_data,
#                              secondary_to_primary_link={"PRODUCT_ID": ['alternatives', 'choice']})

In [7]:
mybt = BradleyTerry(method='BFGS', alpha=1e-5)
mybt.fit_task(products_bought_train)

In [11]:
# find most popular product ids
counter = Counter()
j = []
for i in choice_data.choice.values:
    j = j + [k for k in i]
for z in j:
    counter[z] += 1

print('Most popular product_ids based on amounts purchased')
print(counter.most_common())
print("Bradley Terry rank of items given the covariates")
print(mybt.rank_entities(ascending=False))

Most popular product_ids based on amounts purchased
[(5584007, 626), (5582789, 503), (5585727, 439), (5586076, 331), (951703, 266), (5584100, 248), (849098, 219), (5578643, 210), (6396581, 156), (5591746, 2)]
Bradley Terry rank of items given the covariates
[5584007 5582789 5585727  951703 5586076  849098 5578643 5584100 6396581
 5591746]
